This notebook is based on https://github.com/dmpierre/air-ccs/blob/main/sage/air-to-ccs.ipynb. This notebook implements
- Plonk-to-CCS conversion algorithm, found in the [CCS paper](https://eprint.iacr.org/2023/552), Section 2.2.
- A PoC of incorporating Halo2 lookup into CCS+.
[The explanation of math behind the algorithm is here](https://hackmd.io/@pnyda/SkG0qaUuA)


# The part where we derive c, M, S from custom gates

In [24]:
# Classes to represent a custom gate

from dataclasses import dataclass

@dataclass
class Cell:
    absolute_column_index: int
    relative_row_index: int

@dataclass
class Monomial:
    coefficient: int
    variables: list[Cell]

@dataclass
class CustomGate:
    monomials: list[Monomial]

    def c(self) -> list[int]:
        return [monomial.coefficient for monomial in self.monomials]

    def s(self) -> list[int]:
        i = 0
        
        s = []
        for monomial in self.monomials:
            a = []
            for variable in monomial.variables:
                a.append(i)
                i += 1         
            s.append(a)

        return s

    def t(self) -> list[int]:
        t = 0
        for monomial in self.monomials:
            for variable in monomial.variables:
                t += 1

        return t


p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
F = GF(p)

# def derive_m(table_width: int, table_height: int, gate: CustomGate, copy_constraints: list[(Cell, Cell)]):
def derive_m(table_width: int, table_height: int, gate: CustomGate):
    matrices = [matrix(F, table_height, table_width * table_height) for i in range(gate.t())]

    j = 0
    for monomial in gate.monomials:
        for variable in monomial.variables:
            for y in range(table_height):
                matrices[j][y, variable.absolute_column_index * table_height + (y + variable.relative_row_index) % table_height] = 1
            j += 1

    return matrices
    
            
# a * b + c = d
g_muladd = CustomGate([
    Monomial(1, [Cell(0, 0), Cell(1, 0)]),
    Monomial(1, [Cell(2, 0)]),
    Monomial(-1, [Cell(3, 0)]),
])

print(g_muladd.c())
print(g_muladd.s())

m = derive_m(4, 3, g_muladd)
print("m[0]", m[0])
print("m[1]", m[1])
print("m[2]", m[2])
print("m[3]", m[3])

[1, 1, -1]
[[0, 1], [2], [3]]
m[0] [1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0]
m[1] [0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0]
m[2] [0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0]
m[3] [0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1]


In [19]:
# https://github.com/dmpierre/air-ccs/blob/main/sage/air-to-ccs.ipynb
def ccs_is_satisfied(F, z, matrices, multisets, constants):
    satisfied_instance_witness = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    z_final = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    for i, c in enumerate(constants):
        multiset = multisets[i]
        z_i = vector(F, [1 for i in range(matrices[0].dimensions()[0])])
        for j in multiset:
            z_i = z_i.pairwise_product(matrices[j] * z)
        z_final += c * z_i
    return z_final == satisfied_instance_witness

In [20]:
# Advice columns
table = [[2, 1, 6], [3, 2, 7], [4, 3, 3], [10, 5, 45]]
z = vector(F, table[0] + table[1] + table[2] + table[3])
assert ccs_is_satisfied(F, z, m, g_muladd.s(), g_muladd.c())

In [31]:
# latch(X) * (fib(X) + fib(Xω) - fib(Xω^2)) = 0
g_fib = CustomGate([
    Monomial(1, [Cell(0, 0), Cell(1, 0)]),
    Monomial(1, [Cell(0, 0), Cell(1, 1)]),
    Monomial(-1, [Cell(0, 0), Cell(1, 2)]),
])
m = derive_m(2, 4, g_fib)
print(m[0])
print(m[1])
print(m[2])
print(m[3])
print(m[4])
print(m[5])

# Advice columns
table = [[1, 1, 0, 0], [1, 1, 2, 3]]
z = vector(F, table[0] + table[1])
assert ccs_is_satisfied(F, z, m, g_fib.s(), g_fib.c())

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0]
[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]


In [ ]:
# TODO
- copy constraints
- lookup